# Logistic Regression on imdb for sentiment

https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

In [4]:
import os,tempfile
os.chdir(tempfile.gettempdir())

In [ ]:
import urllib.request,tarfile
urllib.request.urlretrieve('http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz','aclImdb_v1.tar.gz')
tarfile.open('aclImdb_v1.tar.gz','r:gz').extractall()
os.chdir('aclImdb')
cmd='for split in train test; do for sentiment in pos neg; do for file in $split/$sentiment/*; do cat $file >> full_${split}.txt; echo >> full_${split}.txt; done; done; done;'
os.sysmtem(cmd)

In [24]:
import nltk, string
wnl = nltk.WordNetLemmatizer()
reviews_train=[' '.join(wnl.lemmatize(w) for w in nltk.word_tokenize(l) if w.isalpha())
               for l in open('full_train.txt')]
reviews_test=[' '.join(wnl.lemmatize(w) for w in nltk.word_tokenize(l) if w.isalpha())
              for l in open('full_test.txt')]

## BOW - 1-gram

features = 1/0 * vocab size (exist/not exist)

target = 1/0 (positive/negative)


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train)
X = cv.transform(reviews_train)
y = [1 if i < 12500 else 0 for i in range(25000)]

X_test = cv.transform(reviews_test)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

for c in [0.01, 0.03, 0.1, 0.3, 1]:  
    print("Accuracy for C={}: {}".format(c,
                                         np.mean(cross_val_score(LogisticRegression(C=c,solver='liblinear'), X,y, cv=5, scoring='accuracy'))
                                        )
         )

Accuracy for C=0.01: 0.8612399999999999
Accuracy for C=0.03: 0.86492
Accuracy for C=0.1: 0.8626000000000001
Accuracy for C=0.3: 0.85856
Accuracy for C=1: 0.8500400000000001


In [53]:
from sklearn.metrics import accuracy_score
model=LogisticRegression(C=0.03).fit(X,y)
print ("Final Accuracy: %s" % accuracy_score(target, model.predict(X_test)))
feature_to_coef = {
    word: coef for word, coef in zip( cv.get_feature_names(), model.coef_[0] )
}
for best_positive in sorted( feature_to_coef.items(),  key=lambda x: x[1],  reverse=True)[:5]:
    print (best_positive)    
for best_negative in sorted( feature_to_coef.items(),  key=lambda x: x[1])[:5]:
    print (best_negative)

Final Accuracy: 0.87772
('excellent', 0.8593489932761398)
('perfect', 0.6801029769414313)
('great', 0.6545590135908305)
('amazing', 0.5924591149829764)
('favorite', 0.5651963748851384)
('worst', -1.2600324008053674)
('waste', -1.0270444543800354)
('awful', -0.8978979413808582)
('boring', -0.7401303362319944)
('poorly', -0.717995970049884)


## 2-gram?

# Classification by Neural Network

compared with n-gram (counting, then computing MLE).

Now representing words by vec, use CNN along time for look for pattern (larger size = longer-grams).

https://www.kaggle.com/babakgohardani/spam-detection-with-deep-learning-methods

# Language modelling by MLP

# Sequence generation/tagging by RNN